# Gráficas para estudio de reservas en Guatemala

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
reservas = pd.read_excel("db/datos_reservas.xlsx")

In [ ]:
reservas

In [ ]:
plt.figure(figsize=(15, 8))
sns.lineplot(data=reservas.melt('Mes', var_name='Año', value_name='Reservas'), x='Mes', y='Reservas', hue='Año', marker='o', palette=["#19315F", "#F48401", "#BDDB05", "#11C876", "#879484", "#BC2EE6", "#621A22"])

plt.xticks(rotation=45)
plt.title('Reservas Bancarias por Año')
plt.xlabel('Mes')
plt.ylabel('Reservas en millones de quetzales')

# Mueve la leyenda fuera del gráfico, a la parte superior derecha
plt.legend(title='Año', loc='upper left', bbox_to_anchor=(1, 1))

plt.show()

## Calcular variaciones intermensuales e interanuales

In [ ]:
# Variaciones intermensuales
int_mensuales = reservas[[i for i in range(2018, 2024)]].diff().fillna(0)
int_mensuales

In [ ]:
# Hacer diccionario con estadísticos por año
est_anuales = {i: [0,0,0,0] for i in range(2018,2024)}
for i in range(2018, 2024):
    est_anuales[i][0] = reservas[i].mean()
    est_anuales[i][1] = reservas[i].var()
    est_anuales[i][2] = reservas[i].min()
    est_anuales[i][3] = reservas[i].max()

df_est_anuales = pd.DataFrame.from_dict(est_anuales, orient="index", columns=["Promedio", "Varianza", "Mínimo", "Máximo"])
df_est_anuales.index.name = "Año"

df_est_anuales

## Análisis de reservas e inflación

In [101]:
reservas_iniciales_anuales = reservas[reservas["Mes"] == "Enero"]
ritmo_inflacionario = pd.read_excel("db/Ritmo_inflacionario.xlsx")

In [102]:
ritmo_inflacionario

,Año,2018,2019,2020,2021,2022,2023,2024
0,Enero,-0.395434,1.349085,-0.239774,0.154736,-0.039164,0.376457,0.03
1,Febrero,-0.202247,0.136710,-0.402941,0.315712,0.424448,0.636981,0.14
2,Marzo,0.232680,-0.043113,0.482646,0.334806,1.495546,0.378586,NaN
3,Abril,0.044930,0.603839,0.713428,0.106781,0.538151,0.170900,NaN
4,Mayo,0.471557,0.264380,0.189367,0.160000,1.306315,-0.358866,NaN
5,Junio,0.573642,0.826682,1.407070,0.186368,1.830419,0.295212,NaN
6,Julio,0.355556,-0.056545,0.421096,0.332182,1.080981,0.700536,NaN
7,Agosto,0.442870,-0.862801,0.398708,0.205271,0.684429,0.625512,NaN
8,Septiembre,0.595238,-0.584962,0.157480,0.211458,0.358097,0.563528,NaN
9,Octubre,0.336036,0.696039,1.059612,0.362677,0.979740,1.265165,NaN


In [103]:
dic_proyeccion = {i: [None]*12 for i in range(2018, 2024)}
dic_proyeccion["Mes"] = ritmo_inflacionario["Año"].tolist()
df_proyeccion = pd.DataFrame.from_dict(dic_proyeccion)
df_proyeccion = df_proyeccion[["Mes", 2018, 2019, 2020, 2021, 2022, 2023]]

In [104]:
for i in range(2018, 2024):
    for j in range(0, 12):
        if j == 0:
            df_proyeccion.loc[0, i] = reservas.loc[0, i]
        else:
           df_proyeccion.loc[j, i] = df_proyeccion.loc[j-1, i] * (1 + ritmo_inflacionario.loc[j-1,i] / 100)
            

In [105]:
df_proyeccion

,Mes,2018,2019,2020,2021,2022,2023
0,Enero,23583.6,22534.9,25004.7,28095.1,30942.2,36777.0
1,Febrero,23490.342461,22838.914913,24944.745148,28138.573311,30930.081645,36915.449417
2,Marzo,23442.833903,22870.138066,24844.232602,28227.410078,31061.363825,37150.593666
3,Abril,23497.380766,22860.278123,24964.142306,28321.917277,31525.90077,37291.240693
4,Mayo,23507.938223,22998.317327,25142.243484,28352.15958,31695.557742,37354.971377
5,Junio,23618.791524,23059.120309,25189.85469,28397.523036,32109.60154,37220.91718
6,Julio,23754.278893,23249.745876,25544.293667,28450.447067,32697.341762,37330.79767
7,Agosto,23838.738551,23236.599285,25651.859725,28544.954265,33050.793786,37592.313236
8,Septiembre,23944.313124,23036.113775,25754.135649,28603.548728,33277.003081,37827.457484
9,Octubre,24086.838797,22901.361219,25794.693343,28664.033336,33396.166906,38040.625635


In [107]:
# Combinar dfs para poder graficar
df_combinado = reservas.join(df_proyeccion, lsuffix="_re", rsuffix="_pro")
plt.figure(figsize=(10, 12))

for year in range(2018, 2024):  # Asumiendo que tienes datos de 2018 a 2023.
    plt.figure(figsize=(10, 5))  # Crea una nueva figura para cada año.
    plt.plot(df_combinado["Mes_re"], df_combinado[str(year) + '_re'], marker='o', label='Reservas reportadas', color="#FF0000")
    plt.plot(df_combinado["Mes_re"], df_combinado[str(year) + '_pro'], marker='o', linestyle='--', label='Proyección con ritmo inflacionario', color="#0000ff")
    
    plt.title(f'Reservas Bancarias - {year}')
    plt.xlabel('Mes')
    plt.xticks(rotation=25)
    plt.ylabel('Reservas en millones de quetzales')
    plt.legend()
    plt.grid(True)
    
    # Guarda la figura en un archivo.
    # Reemplaza 'ruta/del/directorio' con la ruta real donde deseas guardar los archivos.
    plt.savefig(f'imagenes/reservas_{year}.png', bbox_inches="tight")
    plt.close()  # Cierra la figura para evitar el sobreplotting.

<Figure size 1000x1200 with 0 Axes>